<a href="https://colab.research.google.com/github/mhpbreugem/WP1/blob/main/BBP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
using Distributions, Random, NLsolve, LinearAlgebra, Printf, Optim

In [149]:
# Chapter 0: Parameters

# Economic parameters
β = 0.95
γ = 4.0
e0 = 3.0
EΠ = 1.0
Π0 = 1.0
θXbar = 1.0
θYbar = 1.0
μθX = 0.0
σθX = 0.2
μθY = 0.0
σθY = 0.1
Nσπ = 2
σΠ = 0.4
ρ = 0

# Grid parameters
Nσπ = 21
NNσπ = 1
Nπ = 2 * Nσπ * NNσπ + 1  # number of grid points for payoff

NσΠ = 2
NNσΠ = 1
NΠ = 2 * NσΠ * NNσΠ + 1  # number of grid points for payoff

NθX = 5
NθY = 5
NσθX = 2
NσθY = 2

# Derived midpoints
μπi = (Nπ + 1) ÷ 2
μΠi = (NΠ + 1) ÷ 2
μθXi = (NθX + 1) ÷ 2

# Total number of states
NS = Nπ
NN = NθX * NθY * NΠ

125

In [150]:
# Chapter 1: Grid Construction

# Conjectured Payoffs
ππ = collect(range(EΠ - Nσπ*σΠ, EΠ + Nσπ*σΠ, length=Nπ))
πω = pdf.(Normal(EΠ, σΠ), ππ)
πΩ = πω ./ sum(πω)

# True Payoffs
ΞΠ = EΠ .+ σΠ .* range(-NσΠ, NσΠ, length=NΠ)
ππ_to_index = Dict(round(ππ[πi], digits=8) => πi for πi in 1:Nπ)
ΞΠi = [ππ_to_index[round(ΞΠ[Πi], digits=8)] for Πi in 1:NΠ]

# Signals
S=ππ

function ψϵω(Ю, ϵ)
    σϵ = 1 / sqrt(Ю[1])
    return pdf(Normal(0.0, σϵ), ϵ)
end

function φSΩπ(Ю)
    σϵ = 1 / sqrt(Ю[1])
    SΩ = [pdf(Normal(π, σϵ), s) for π in ππ, s in ππ]
    SΩnorm = SΩ ./ sum(SΩ, dims=2)
    return SΩnorm
end

# Noise Trader Demand
ΞθX = μθX .+ σθX .* range(-NσθX, NσθX, length=NθX)
ΞθY = μθY .+ σθY .* range(-NσθY, NσθY, length=NθY)

function θXYωω(x, y)
    Σθ = [σθX^2  σθX*σθY*ρ;
          σθX*σθY*ρ  σθY^2]
    d = MvNormal([μθX, μθY], Σθ)
    return pdf(d, [x, y])
end

θXYωω (generic function with 1 method)

In [151]:
# Chapter 2: Learning

# Full learning (private + public information)
function φΞΩI(Ю, Ж)
    XΣ = Ж[2*NS+1:2*NS+Nπ]
    YΣ = Ж[2*NS+Nπ+1:2*NS+2*Nπ]

    ϵ = S .- ππ'
    ψϵ_values = ψϵω.(Ю, ϵ)
    θXY_values = θXYωω.(θXbar .- XΣ, θYbar .- YΣ)

    ΞΩI = πΩ .* ψϵ_values .* θXY_values
    ΞΩI .= ΞΩI ./ sum(ΞΩI, dims=1)

    return ΞΩI
end

φΞΩI (generic function with 1 method)

In [153]:
# Chapter 3: First-Order Conditions

# Fundamental
Δ1 = Π0
Δ2 = ππ

# Initial Wealth
W0 = e0

# FOC's
function φFOC(Ж, Ω)
    X = Ж[1:NS]
    C1 = Ж[NS+1:2*NS]
    PX, PY = Ж[2*NS+2*Nπ+1:2*NS+2*Nπ+2]

    C2_values = (W0 .- PX .* X' .- C1') ./ PY .+ X' .* Δ2
    FOCX_values = β .* sum(Ω .* exp.(-γ .* C2_values) .* (-PX ./ PY .+ Δ2), dims=1)
    FOCC1_values = exp.(-γ .* C1') + β .* sum(Ω .* exp.(-γ .* C2_values) .* (-PX ./ PY .+ Δ2), dims=1)

    return FOCX_values, FOCC1_values
end


φFOC (generic function with 1 method)

In [154]:
# Chapter 4: Aggregation

# Total initial wealth
ΣW0 = e0

# Aggregate Demand
function φAD(Ж, Ю)
    X = Ж[1:NS]
    C1 = Ж[NS+1:2*NS]
    PX, PY = Ж[2*NS+2*Nπ+1:2*NS+2*Nπ+2]
    XΣ = Ж[2*NS+1:2*NS+Nπ]
    YΣ = Ж[2*NS+Nπ+1:2*NS+2*Nπ]

    SΩπ = φSΩπ(Ю)

    SΩ_X_sum = sum(SΩπ .* X', dims=2)
    XiADX_values = XΣ .- SΩ_X_sum

    SΩ_C1_sum = sum(SΩπ .* C1', dims=2)
    XiADY_values = YΣ .* PY .- (ΣW0 .- PX .* XΣ .- SΩ_C1_sum)

    return XiADX_values, XiADY_values
end

# Market Clearing
function φMC(Ж, Б)
    XΣ = Ж[2*NS+1:2*NS+Nπ]
    YΣ = Ж[2*NS+Nπ+1:2*NS+2*Nπ]

    Πi = Б[1]
    θX = ΞθX[Б[2]]
    θY = ΞθY[Б[3]]

    MCX = XΣ[ΞΠi[Πi]] - (θXbar - θX)
    MCY = YΣ[ΞΠi[Πi]] - (θYbar - θY)

    return MCX, MCY
end

φMC (generic function with 1 method)

In [198]:
# Chapter 5: Zero-Info Starting Point (ξ)
W0ξ = e0

# Fixed Point Zero info
function φSYSξ(Жξ,Б)
    # Unpack parameters
    PXξ, PYξ = Жξ[1:2]
    θX = ΞθX[Б[2]]
    θY = ΞθY[Б[3]]

    # Direct solution of MC's
    Xξ = (θXbar - θX)
    C1ξ = W0ξ - PXξ * (θXbar - θX) - PYξ *(θYbar - θY)

    # Compute second-period consumption C2ξ[πi, Si]
    C2ξ_values = (W0ξ - PXξ * Xξ - C1ξ) / PYξ  .+ Xξ .* Δ2

    # FOC's
    FOCXξ_values = β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-PXξ ./ PYξ .+ Δ2), dims=1)
    FOCC1ξ_values = exp.(-γ .* C1ξ) .+ β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-1.0 ./ PYξ), dims=1)

    return vcat(FOCXξ_values, FOCC1ξ_values)
end

# Starting Point Full Code
function φЖ0(Б)
    θXi, θYi = Б[2], Б[3]
    θX = ΞθX[θXi]
    θY = ΞθY[θYi]
    solξ = nlsolve(Жξ -> φSYSξ(Жξ, [0, θXi, θYi]), [0.6, 0.7])
    PXξ, PYξ = solξ.zero
    Ж = zeros(2 * NS + 2 * Nπ + 2)
    Ж[1:NS] .= θXbar - θX
    Ж[NS+1:2*NS] .= W0ξ .- PXξ * (θXbar .- θX) .- PYξ * (θYbar .- θY)
    Ж[2*NS+2*Nπ+1:2*NS+2*Nπ+2] .= [PXξ, PYξ]
    Ж[2*NS+1:2*NS+Nπ] .= θXbar .- θX
    Ж[2*NS+Nπ+1:2*NS+2*Nπ] .= θYbar .- θY
    return Ж
end


φЖ0 (generic function with 1 method)

φЖ0 (generic function with 1 method)

In [199]:
@show φЖ0([0,4,3])'

(φЖ0([0, 4, 3]))' = [0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635133042043 1.6082635

1×174 adjoint(::Vector{Float64}) with eltype Float64:
 0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.8  …  1.0  1.0  1.0  0.488469  1.00096

In [157]:
Жξ_ini = [0.7,0.6]

solξ = nlsolve(Жξ -> φSYSξ(Жξ,[0,3,3]), Жξ_ini)

Жξ_star = solξ.zero

2-element Vector{Float64}:
 0.3482193243636204
 0.9672753330412129

In [78]:
φSYSξ([1,1,1,1],[1,1,1])

([-119.3928845327993], [-186.5330641396361], -0.3999999999999999, -3.2)

In [21]:
# Chapter 6: Fixed Point

# System of equations
function φSYS(Ж, Ю, Б)
    Ω = φΞΩI(Ю, Ж)
    FOCX, FOCC1 = φFOC(Ж, Ω)
    XiADX, XiADY = φAD(Ж, Ю)
    MCX, MCY = φMC(Ж, Б)

    return vcat(vec(FOCX), vec(FOCC1), vec(XiADX), vec(XiADY), MCX, MCY)
end

# Starting Point
    X_init = 0.0 .* ones(NS)
    C1_init = (e0 / 1) .* ones(NS)
    XΣ_init = 0.0 .* ones(Nπ)
    YΣ_init = 0.0 .* ones(Nπ)
    PX_init = 1.0
    PY_init = 1.0

    Ж_init = vcat(X_init, C1_init, XΣ_init, YΣ_init, [PX_init, PY_init])

174-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0

In [31]:
Ю_fix = [0.1]
Б_fix = (1, 3, 3)

function SYS_only_Ж(Ж)
    return φSYS(Ж, Ю_fix, Б_fix)
end

sol = nlsolve(SYS_only_Ж, Ж_init)

Ж_star = sol.zero

println("Solution Ж*: ", Ж_star)
println("Residuals at solution: ", φSYS(Ж_star, Ю_fix, Б_fix))


Solution Ж*: [0.808868516120515, 0.8188684869168946, 0.8288684598327343, 0.8388684344388493, 0.8488684105586516, 0.8588683882818502, 0.8688683679772442, 0.8788683503125186, 0.8888683362062448, 0.8988683267152013, 0.9088683228778112, 0.9188683255554281, 0.9288683352417276, 0.9388683518594587, 0.9488683746130832, 0.958868401964927, 0.9688684317528926, 0.9788684614202421, 0.9888684883170312, 0.9988685099885076, 1.008868524325104, 1.0188685303596776, 1.028868527623012, 1.0388685165964593, 1.0488684984692787, 1.0588684748805277, 1.0688684483009068, 1.0788684214291637, 1.0888683968318669, 1.0988683767054672, 1.1088683626391531, 1.118868355486916, 1.128868355375711, 1.1388683618264634, 1.1488683739546097, 1.158868390705258, 1.1688684110666334, 1.1788684342083733, 1.1888684595603376, 1.1988684868434596, 1.2088685160734947, 1.2188685475524217, 1.2288685818637166, 9.37740624471356, 9.371301668423452, 9.365017012164557, 9.358226231221584, 9.350552145254266, 9.341887561776746, 9.33332437028789, 9.

In [23]:

Ю_fix = [0.2]
Б_fix = (1, 3, 3)

function SYS_only_Ж(Ж)
    return φSYS(Ж, Ю_fix, Б_fix)
end

sol = nlsolve(SYS_only_Ж, Ж_star; method=:trust_region, store_trace=true, show_trace=true)

Ж_star = sol.zero

println("Solution Ж*: ", Ж_star)
println("Residuals at solution: ", φSYS(Ж_star, Ю_fix, Б_fix))


LoadError: UndefVarError: `Ж_star` not defined



---



In [24]:
# Set a fixed seed for reproducibility
Random.seed!(1234)

# Generate a random vector Ж with mean 0.1, std 0.01, and length 177
size = 176
Ж = rand(1, size)
println("Ж = ", Ж)

Ω = φΞΩI(0.1, Ж)

Ж = [0.5798621201341324 0.4112941179498505 0.9721360824554687 0.014908849285099945 0.520354993723718 0.6395615996802734 0.8396219340580711 0.967142768915383 0.7897644095351307 0.6960406981439002 0.5667043025437501 0.5363685687293304 0.7113892493371533 0.1039294604697858 0.806704115694322 0.8705393388965176 0.9627145232215633 0.15118014494810728 0.7153546713317517 0.9395476419202511 0.5263436792092471 0.07796832278241894 0.9661966846171819 0.6665578560615069 0.33385905430793017 0.8021987716647205 0.15645840849748271 0.3807992099077905 0.06875911697436776 0.8617248852475133 0.00543655955195288 0.6984777190448724 0.3492308989506544 0.9109702293941395 0.9736533335143757 0.7606595293677095 0.4889116091598197 0.36264257055837246 0.7087378217590448 0.6566936421651552 0.1738629177690535 0.6973601779506733 0.19172328131691596 0.10051452541181105 0.4213882946411107 0.8159260441267123 0.9439639524152046 0.7392551298618548 0.01775088258636426 0.7749752377279369 0.2164667906754636 0.514119270588462

43×43 Matrix{Float64}:
 4.48661e-112  3.2615e-112   2.37089e-112  …  9.29634e-118  6.75426e-118
 5.143e-89     3.79896e-89   2.80612e-89      2.05356e-94   1.51608e-94
 2.69049e-80   2.01943e-80   1.51572e-80      2.07023e-85   1.55304e-85
 1.92324e-76   1.46683e-76   1.11871e-76      2.85179e-81   2.17385e-81
 2.42409e-80   1.87864e-80   1.4559e-80       6.92677e-85   5.36527e-85
 2.5497e-58    2.00785e-58   1.58113e-58   …  1.404e-62     1.10504e-62
 2.58558e-51   2.06895e-51   1.65552e-51      2.74369e-55   2.19429e-55
 2.07441e-47   1.68669e-47   1.37142e-47      4.24199e-51   3.44728e-51
 1.26636e-42   1.04628e-42   8.64427e-43      4.99031e-46   4.12082e-46
 9.70991e-32   8.1518e-32    6.8436e-32       7.37367e-35   6.18712e-35
 2.37117e-30   2.02278e-30   1.72555e-30   …  3.46999e-33   2.95856e-33
 2.7164e-22    2.35467e-22   2.04107e-22      7.6605e-25    6.6368e-25
 6.38612e-35   5.62499e-35   4.95449e-35      3.47054e-37   3.05526e-37
 ⋮                                       

In [25]:
Ω = φΞΩI(0.1, fill(0.1, 176))
φFOCX(fill(0.1, 176), Ω)

LoadError: UndefVarError: `φFOCX` not defined

In [26]:
function compute_PXPY_plus_e0(Ж::Vector{Float64}, Ю::Vector{Float64}, NS::Int, NΠ::Int)
    # PX is at position 2NS + 2NΠ + 1
    idx_PX = 2*NS + 2*NΠ + 1
    idx_PY = 2*NS + 2*NΠ + 2

    PX = Ж[idx_PX]
    PY = Ж[idx_PY]

    return PX * PY + e0
end

# Call the function
result = compute_PXPY_plus_e0(Ж, Ю, NS, NΠ)
println("PX × PY + e0 = ", result)


LoadError: UndefVarError: `Ю` not defined